In [1]:
##두바이유
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 기본 URL
base_url = "https://finance.naver.com/marketindex/worldDailyQuote.naver"

# 크롤링 결과 저장 리스트
results = []

# 페이지 범위 지정 (예: 1페이지부터 5페이지까지)
for page in range(1, 6):
    # URL에 page 파라미터 추가
    params = {
        "marketindexCd": "OIL_DU",  # 두바이유
        "fdtc": 2,  # 추가 파라미터
        "page": page
    }

    # 요청 보내기
    response = requests.get(base_url, params=params)
    response.raise_for_status()

    # BeautifulSoup으로 파싱
    soup = BeautifulSoup(response.text, "html.parser")

    # 테이블에서 행 데이터 찾기
    rows = soup.select("table.tbl_exchange tbody tr")

    for row in rows:
        # 각 열 데이터 추출
        cols = row.find_all("td")
        if len(cols) == 0:
            continue

        date = cols[0].text.strip()  # 날짜
        price = cols[1].text.strip().replace(",", "")  # 쉼표 제거 후 가격

        # 결과 리스트에 추가
        results.append({
            "날짜": date,
            "가격": price
        })

# DataFrame으로 변환
df = pd.DataFrame(results)

# 날짜를 시간 순으로 정렬 (최신 날짜가 위로 오게 변경)
df["날짜"] = pd.to_datetime(df["날짜"], format="%Y.%m.%d")
df["가격"] = pd.to_numeric(df["가격"])
df = df.sort_values("날짜", ascending=False)  # 최신 날짜가 위로 오도록 정렬

# 결과 확인
print(df)

# CSV로 저장
output_file = "dubai_crude_prices.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"크롤링 결과가 {output_file} 파일에 저장되었습니다.")


           날짜     가격
0  2025-01-13  80.07
1  2025-01-10  78.76
2  2025-01-09  76.78
3  2025-01-08  76.28
4  2025-01-07  76.99
5  2025-01-06  76.51
6  2025-01-03  76.67
7  2025-01-02  76.14
8  2024-12-31  74.76
9  2024-12-30  74.08
10 2024-12-27  73.73
11 2024-12-26  72.92
12 2024-12-24  73.24
13 2024-12-23  72.73
14 2024-12-20  72.79
15 2024-12-19  72.76
16 2024-12-18  72.94
17 2024-12-17  72.85
18 2024-12-16  73.13
19 2024-12-13  73.41
20 2024-12-12  72.79
21 2024-12-11  72.80
22 2024-12-10  71.94
23 2024-12-09  71.96
24 2024-12-06  71.16
25 2024-12-05  71.79
26 2024-12-04  71.96
27 2024-12-03  73.07
28 2024-12-02  71.35
29 2024-11-29  71.36
30 2024-11-27  71.78
31 2024-11-26  71.70
32 2024-11-25  71.73
33 2024-11-22  73.06
34 2024-11-21  72.83
크롤링 결과가 dubai_crude_prices.csv 파일에 저장되었습니다.


In [7]:
#다른 링크, WRI
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# URL 및 드라이버 설정
url = "https://m.stock.naver.com/marketindex/energy/CLcv1"
driver = webdriver.Chrome()  # 크롬 드라이버 사용

# 페이지 열기
driver.get(url)
time.sleep(2)  # 페이지 로드 대기

# 데이터 저장 리스트
results = []

# "더보기" 버튼 클릭 3번
for _ in range(3):
    try:
        # "더보기" 버튼이 나타날 때까지 대기
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "InfinityMoreButton_button__7qjxe"))
        )
        more_button.click()  # 버튼 클릭
        time.sleep(2)  # 데이터 로드 대기
    except Exception as e:
        print(f"더보기 버튼을 클릭하는 중 오류 발생: {e}")
        break

# 데이터 크롤링
table_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
for row in table_rows:
    try:
        cols = row.find_elements(By.TAG_NAME, "td")
        date = cols[0].text.strip()  # 날짜
        close_price = cols[1].text.strip()  # 종가
        change = cols[2].text.strip()  # 전일대비
        change_rate = cols[3].text.strip()  # 등락률

        # 결과 리스트에 추가
        results.append({
            "날짜": date,
            "종가": close_price,
            "전일대비": change,
            "등락률": change_rate
        })
    except IndexError:
        continue  # 데이터가 부족한 행은 건너뛰기

# Selenium 브라우저 닫기
driver.quit()

# DataFrame으로 변환
df = pd.DataFrame(results)

# 날짜 형식 보완
current_year = pd.Timestamp.now().year  # 현재 연도 가져오기
df["날짜"] = df["날짜"].apply(lambda x: f"{current_year}.{x.strip()}" if len(x.strip().split('.')) == 2 else x)

# 날짜 형식 변환
df["날짜"] = pd.to_datetime(df["날짜"], format="%Y.%m.%d", errors="coerce")

# 변환되지 않은 날짜 확인 (필요 시 처리)
invalid_dates = df[df["날짜"].isna()]
if not invalid_dates.empty:
    print("변환되지 않은 날짜 데이터:")
    print(invalid_dates)

# 날짜 정렬 (최신 날짜가 위로)
df = df.sort_values("날짜", ascending=False)

# 결과 출력
print(df)


# CSV로 저장
output_file = "crude_oil_data.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"크롤링 결과가 {output_file} 파일에 저장되었습니다.")




변환되지 않은 날짜 데이터:
    날짜     종가      전일대비     등락률
0  NaT  78.82  상승\n2.25  +2.94%
1  NaT  76.57  상승\n2.65  +3.59%
2  NaT  73.92  상승\n0.60  +0.82%
3  NaT  73.32  하락\n0.93  -1.25%
4  NaT  74.25  상승\n0.69  +0.94%
5  NaT  73.56  하락\n0.40  -0.54%
6  NaT  73.96  상승\n0.83  +1.14%
7  NaT  73.13  상승\n1.41  +1.97%
8  NaT  71.72  상승\n0.73  +1.03%
9  NaT  70.99  상승\n0.39  +0.55%
10 NaT  70.60  상승\n0.98  +1.41%
11 NaT  69.62  하락\n0.48  -0.68%
12 NaT  70.10  상승\n0.86  +1.24%
13 NaT  69.24  하락\n0.22  -0.32%
14 NaT  69.46  상승\n0.08  +0.12%
15 NaT  69.38  하락\n0.64  -0.91%
16 NaT  70.02  상승\n0.37  +0.53%
17 NaT  70.08  하락\n0.63  -0.89%
18 NaT  70.71  하락\n0.58  -0.81%
19 NaT  71.29  상승\n1.27  +1.81%
20 NaT  70.02  하락\n0.27  -0.38%
21 NaT  70.29  상승\n1.70  +2.48%
22 NaT  68.59  상승\n0.22  +0.32%
23 NaT  68.37  상승\n1.17  +1.74%
24 NaT  67.20  하락\n1.10  -1.61%
25 NaT  68.30  하락\n0.24  -0.35%
26 NaT  68.54  하락\n1.40  -2.00%
27 NaT  69.94  상승\n1.84  +2.70%
28 NaT  68.10  상승\n0.10  +0.15%
29 NaT  68.00  하락\n0.72 